In [4]:
from keras_facenet import FaceNet
import matplotlib.pyplot as plt
import numpy as np 
import os
import tensorflow as tf

embedder = FaceNet()

In [5]:
anchor_path = 'people/'

In [6]:

anchors = os.listdir(anchor_path)
embeddings = []
names = []
for image_name in anchors:
    image = plt.imread(anchor_path + image_name)
    faces = embedder.extract(image, threshold = 0.96)
#     print(faces[0].keys())
#     x1, y1, w, h = faces[0]['box']
#     x2, y2, = x1 + w, y1 + h
    try:
#         cropped_image = image[y1:y2, x1:x2]
#         plt.imshow(cropped_image)
#         plt.show()
        embeddings.append(np.squeeze(faces[0]['embedding']))
        names.append(image_name.split('.')[0])
    except:
        pass
            

In [8]:
import pandas as pd
names_df = pd.DataFrame(names, columns = ['Name'])
embeddings_df = pd.DataFrame(embeddings)
final_df = pd.concat([names_df,embeddings_df], axis = 1)
final_df.head(5)

In [9]:
final_df.to_csv('Encodings.csv', index = False)

In [10]:
import pandas as pd
Encodings_df = pd.read_csv('Encodings.csv')
Encodings_df.head()

In [4]:

def compare(v1):
    min_distance = 1000000.0
    final_name = ''
    for i in range(len(Encodings_df)):
        row = Encodings_df.iloc[i,]
        name = row['Name']
        v2 = row[Encodings_df.columns[1:]]
        dist = np.linalg.norm(v1-v2)
        if dist < min_distance:
            min_distance = dist
            final_name = name
    return final_name

    
def identify(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    names = []
    faces = embedder.extract(image, threshold = 0.97)
    for face in faces:
        if face['confidence'] >= 0.97:
            x1, y1, w, h = face['box']
            x2, y2 = x1+w, y1+h
            plt.plot([x1,x2,x2,x1,x1],[y1,y1,y2,y2,y1])
            encoding = face['embedding']
            name = compare(encoding)
            plt.text(x1,y1, name, bbox = dict(fill = True, linewidth = 1))
            names.append(name)
    plt.show()
    plt.close()
    return names
            

In [11]:
# Run this cell to take picture from webcam and recognise Face
import cv2, time

camera = cv2.VideoCapture(0)
time.sleep(1)
check, image = camera.read()
time.sleep(1)
if check:
    names = identify(image)
camera.release()
        

In [12]:
test_path = 'test/'
for name in os.listdir(test_path):
    image = plt.imread(test_path + name)
    names = identify(image)
#     print(names)

In [13]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
x = Encodings_df.iloc[:,np.arange(1,511)].values
x.head()

In [14]:
principal_components = pca.fit_transform(x)
principal_df = pd.DataFrame(principal_components, columns = ['component_1', 'component_2'])
principal_df

In [15]:
xs = principal_df['component_1']
ys = principal_df['component_2']

names = Encodings_df['Name']
colors = np.arange(0,len(xs))
for i in range(len(xs)):
    plt.scatter([xs[i]], [ys[i]], label = names[i])

plt.legend()
plt.show()